In [1]:
# importing required modules
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import LineTokenizer
import glob
import re
import os
import pandas as pd
import numpy as np
import torch
from torch.cuda import profiler
import cProfile
from itertools import chain
from sentence_transformers import SentenceTransformer
import time

#pandas max columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

In [2]:
start_time = time.time()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU instead.")

No GPU available, using CPU instead.


In [4]:
# # start the profiler
# profiler.start()

In [5]:
# change the File directory
file_list = glob.glob(os.path.join(os.getcwd(), "C:/Users/me1awq/PhD/docsim/datasets/rssb", "*.csv"))
corpus = []
files = []

for file_path in file_list:
    with open(file_path, encoding="latin-1") as f_input:
        corpus.append(f_input.read())
        files.append(''.join([n for n in os.path.basename(file_path)]))

main_df = pd.DataFrame({'Document name':files, 'Document text':corpus})
main_df

Document name  \
0                   EA rulebook.csv   
1         EA_responsible_person.csv   
2                  mpnet_result.csv   
3                  OCD rulebook.csv   
4  OCDlookup_responsible_person.csv   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [6]:
# importing the stopwords using nltk library.
from nltk.corpus import stopwords
stop = stopwords.words('english')

custom_stopwords = ["ï","»","¿","â€“","â€™","a)","â","¿","–","–","b)","c)","d)","e)",":","(",")","â€˜","-",'must','used','using'
                   'near']
# punctation=[":","(",")"]
stop.extend(custom_stopwords)
# stop.extend(punctation)

In [7]:
main_df['Clean_documents_rules'] = main_df['Document text'].str.lower()
#main_df['Clean_documents_rules']= main_df['Clean_documents_rules'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
main_df

Document name  \
0                   EA rulebook.csv   
1         EA_responsible_person.csv   
2                  mpnet_result.csv   
3                  OCD rulebook.csv   
4  OCDlookup_responsible_person.csv   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [8]:
main_df['Clean_documents_rules'] = main_df['Document text'].str.lower()
#main_df['Clean_documents_rules']= main_df['Clean_documents_rules'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
main_df

Document name  \
0                   EA rulebook.csv   
1         EA_responsible_person.csv   
2                  mpnet_result.csv   
3                  OCD rulebook.csv   
4  OCDlookup_responsible_person.csv   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [9]:
def token_sent(text):   
    sent_tokens=LineTokenizer(blanklines='keep').tokenize(text)
    return sent_tokens

In [10]:
main_df['Sentence_Tokenize_rules']=main_df['Clean_documents_rules'].apply(token_sent) 
main_df['Word_Tokenize_rules']=main_df['Clean_documents_rules'].apply(word_tokenize) 
main_df

Document name  \
0                   EA rulebook.csv   
1         EA_responsible_person.csv   
2                  mpnet_result.csv   
3                  OCD rulebook.csv   
4  OCDlookup_responsible_person.csv   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [11]:
token_doc_name = dict(zip(main_df['Document name'], main_df['Sentence_Tokenize_rules']))
token_doc_name

{'EA rulebook.csv': ['driver',
  'driver',
  'driver',
  'im',
  'im',
  'im',
  'im',
  'maintenance',
  'maintenance',
  'maintenance',
  'public',
  'public',
  'public',
  'railway workers',
  'railway workers',
  'railway workforce',
  'railway workforce',
  'train crew',
  'train crew',
  'train controller',
  'train operating staff',
  '2.1.1 - context',
  '2.1.1.1 - establish spatial envelope around each train in one section of line at a time',
  '2.1.1.2 - space interval effective against collision',
  '"2.1.1.3 - space interval reduced or eliminated for permissive working or train failure, but not coupling or uncoupling"',
  '2.1.1.4 - written operating procedures or spoken instructions when the signalling system cannot be used to preserve the effectiveness of the space interval',
  '2.1.1.5 - authority of train movements under normal or degraded conditions',
  '2.1.2 - control operation under normal conditions',
  '2.1.2.1 - im authorises permissive working',
  '2.1.2.2 cont

# Document similarity

In [12]:
def cosine_sim(embeddings1, embeddings2):
    """Find the cosine similarity distance between two vectors."""
    # convert embeddings1 and embeddings2 to PyTorch tensors
    embeddings1 = torch.tensor(embeddings1)
    embeddings2 = torch.tensor(embeddings2)
    
    # calculate dot product using PyTorch's matmul() function
    dot_product = torch.matmul(embeddings1, embeddings2.T)
    
    # calculate the norm using PyTorch's norm() function
    norm1 = torch.norm(embeddings1)
    norm2 = torch.norm(embeddings2)
    
    # calculate the cosine similarity
    csim = dot_product / (norm1 * norm2)
    
    # check for NaN values
    if torch.isnan(torch.sum(csim)):
        return 0
    
    return csim.item()

In [13]:
def find_similar_sentences(search_sentence, model, main_df, threshold=0.5):
    #embed the search sentence
    search_sentence_embeddings = model.encode(search_sentence)

    docs_sent_tokens = list(chain.from_iterable(main_df['Sentence_Tokenize_rules']))

    results=[]

    #embedding all the documents and find the similarity between search text and all the tokenize sentences
    for doc_name, docs_sent_tokens in token_doc_name.items():
        name = doc_name.title()
        for docs_sent_token in docs_sent_tokens:
            sentence_embeddings = model.encode(docs_sent_token)
            sim_score1 = cosine_sim(search_sentence_embeddings, sentence_embeddings)
            if sim_score1 > threshold:
                results.append((
                    docs_sent_token,
                    sim_score1,
                    name
                ))

    #printing the top 10 matching result in dataframe format
    df=pd.DataFrame(results, columns=['Matching Sentence','Similarity Score','Document name'])

    # sorting in descending order based on the similarity score
    df.sort_values("Similarity Score", ascending = False, inplace = True)

    #change the value of n to see more results
    return df.head(n=10)



In [14]:
# assuming you have already defined search_sentence, model, and main_df variables
similar_sentences = find_similar_sentences(search_sentence="2.6.2.4 Operating rules must include:", model = SentenceTransformer('sentence-transformers/msmarco-distilroberta-base-v2'), main_df=main_df, threshold=0.7)

# Create a pandas DataFrame from the function results
similar_sentences_df = pd.DataFrame(similar_sentences)

# Print the DataFrame
similar_sentences_df

C:\Users\me1awq\AppData\Local\Temp\ipykernel_108256\3657401743.py:8: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3281.)
  dot_product = torch.matmul(embeddings1, embeddings2.T)


,Matching Sentence,Similarity Score,Document name
3,2.3.1.4 operating rules are used:,0.884548,Ocd Rulebook.Csv
0,2.6.2.4 operating rules includes vehicle and infrastructure constraints and procedures,0.810288,Ea Rulebook.Csv
4,2.8.3.4 operating rules must include procedures to be applied by people working alone.,0.755362,Ocd Rulebook.Csv
2,"""2.2.2.4 operating rules must include protocols for clear communication between all those involved in procedures to authorise train movements, using spoken or written instructions.""",0.731789,Ocd Rulebook.Csv
1,"""2.1.3.3 operating rules must be supported by a planned organisational response from the infrastructure manager, to mobilise the resources needed to implement the procedures.""",0.731675,Ocd Rulebook.Csv


In [15]:
end_time = time.time()

# calculate the total time taken to execute the code on the GPU
total_time = end_time - start_time

print(f"Total time taken to execute the code on CPU: {total_time:.6f} seconds")

Total time taken to execute the code on CPU: 40.584056 seconds


In [16]:
# # profile the function
# cProfile.run('cosine_sim(embeddings1, embeddings2)')

In [17]:
# # profile the function
# cProfile.run('similar_sentences = find_similar_sentences(search_sentence="2.6.2.4 Operating rules must include:", model = SentenceTransformer('sentence-transformers/msmarco-distilroberta-base-v2'), main_df=main_df, threshold=0.7)')